In [27]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime
from dotenv import load_dotenv
import os

In [28]:
load_dotenv()
API_KEY_WAPI = os.getenv("API_KEY_WAPI")
PHONE_NUMBER = os.getenv("PHONE_NUMBER")
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")
TWILIO_SID = os.getenv("TWILIO_SID")
MY_PHONE_NUMBER=os.getenv("MY_PHONE_NUMBER")

In [29]:
place = "Corrientes, Corrientes"
api_key = API_KEY_WAPI
url_base ="http://api.weatherapi.com/v1/forecast.json?key="+api_key+"&q="+place+"&days=1&aqi=no&alerts=no"

In [30]:
response = requests.get(url_base).json()


In [31]:
response

{'location': {'name': 'Corrientes',
  'region': 'Corrientes',
  'country': 'Argentina',
  'lat': -27.4667,
  'lon': -58.8333,
  'tz_id': 'America/Argentina/Cordoba',
  'localtime_epoch': 1768413351,
  'localtime': '2026-01-14 14:55'},
 'current': {'last_updated_epoch': 1768412700,
  'last_updated': '2026-01-14 14:45',
  'temp_c': 33.2,
  'temp_f': 91.8,
  'is_day': 1,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
   'code': 1003},
  'wind_mph': 6.3,
  'wind_kph': 10.1,
  'wind_degree': 37,
  'wind_dir': 'NE',
  'pressure_mb': 1010.0,
  'pressure_in': 29.83,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 49,
  'cloud': 75,
  'feelslike_c': 34.7,
  'feelslike_f': 94.4,
  'windchill_c': 33.8,
  'windchill_f': 92.8,
  'heatindex_c': 35.7,
  'heatindex_f': 96.2,
  'dewpoint_c': 18.1,
  'dewpoint_f': 64.6,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 13.6,
  'gust_mph': 7.2,
  'gust_kph': 11.6},
 'forecast': {'forecastday': [{'d

In [32]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [33]:
response["forecast"]["forecastday"][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [34]:
len(response["forecast"]["forecastday"][0]["hour"])

24

In [35]:
response["forecast"]["forecastday"][0]["hour"][0]["time"].split()[0]

'2026-01-14'

In [36]:
int(response["forecast"]["forecastday"][0]["hour"][0]["time"].split()[1].split(":")[0])

0

In [37]:
response["forecast"]["forecastday"][0]["hour"][0]["condition"]["text"]

'Clear '

In [38]:
response["forecast"]["forecastday"][0]["hour"][0]["temp_c"]

23.5

In [39]:
response["forecast"]["forecastday"][0]["hour"][0]["will_it_rain"]

0

In [40]:
response["forecast"]["forecastday"][0]["hour"][0]["chance_of_rain"]

0

DATAFRAME

In [41]:
def getforecast(response,i):
    fecha= response["forecast"]["forecastday"][0]["hour"][i]["time"].split()[0]
    hora= int(response["forecast"]["forecastday"][0]["hour"][i]["time"].split()[1].split(":")[0])
    condicion= response["forecast"]["forecastday"][0]["hour"][i]["condition"]["text"]
    temp_c= response["forecast"]["forecastday"][0]["hour"][i]["temp_c"]
    rain= response["forecast"]["forecastday"][0]["hour"][i]["will_it_rain"]
    chance_rain= response["forecast"]["forecastday"][0]["hour"][i]["chance_of_rain"]

    return [fecha,hora,condicion,temp_c,rain,chance_rain]

In [42]:
datos=[]
for i in tqdm(range(len(response["forecast"]["forecastday"][0]["hour"])),colour="red"):
    datos.append(getforecast(response,i))

100%|██████████| 24/24 [00:00<?, ?it/s]


In [43]:
datos[0]

['2026-01-14', 0, 'Clear ', 23.5, 0, 0]

In [44]:
col=["fecha","hora","condicion","temp_c","rain","chance_rain"]
Df=pd.DataFrame(datos,columns=col)

In [45]:
Df

,fecha,hora,condicion,temp_c,rain,chance_rain
0,2026-01-14,0,Clear,23.5,0,0
1,2026-01-14,1,Clear,23.0,0,0
2,2026-01-14,2,Clear,22.7,0,0
3,2026-01-14,3,Clear,22.3,0,0
4,2026-01-14,4,Partly Cloudy,22.0,0,0
5,2026-01-14,5,Partly Cloudy,21.7,0,0
6,2026-01-14,6,Clear,21.3,0,0
7,2026-01-14,7,Partly Cloudy,21.3,0,0
8,2026-01-14,8,Overcast,23.4,0,0
9,2026-01-14,9,Overcast,25.9,0,0


In [46]:
df_rain=Df[(Df["rain"]==1) | (Df["chance_rain"]>50)]
df_rain=df_rain[["hora","condicion"]]
df_rain.set_index("hora",inplace=True)

In [47]:
df_rain

,condicion
hora,


In [48]:
if df_rain.empty:
    mensaje = (
        f"\nHola! \n\n\n El pronóstico de hoy {Df['fecha'][0]} en {place} indica que no hay horas con probabilidad de lluvia.\n\n\n Que tengas un buen día! :)"
    )
else:
    mensaje = (
        f"\nHola! \n\n\n El pronóstico de hoy {Df['fecha'][0]} en {place} indica que hay:\n\n"
        f"{len(df_rain)} horas con probabilidad de lluvia:\n\n{df_rain.to_string()}\n\n\n Que tengas un buen día! :)"
    )

In [49]:
mensaje

'\nHola! \n\n\n El pronóstico de hoy 2026-01-14 en Corrientes, Corrientes indica que no hay horas con probabilidad de lluvia.\n\n\n Que tengas un buen día! :)'

In [ ]:
#!pip install twilio

In [50]:
import time
from twilio.rest import Client

In [51]:
import time
from twilio.rest import Client
time.sleep(2)
client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)
print(f"Enviando desde: {PHONE_NUMBER} a: {MY_PHONE_NUMBER}")
print(f"Mensaje: {mensaje}")
try:
    message = client.messages.create(
        body=mensaje,
        from_=PHONE_NUMBER,
        to=MY_PHONE_NUMBER
    )
    print("mensaje enviado " + message.sid)
except Exception as e:
    print("Error al enviar el mensaje:", e)

Enviando desde: +1 570 715 6604 a: +543794146610
Mensaje: 
Hola! 


 El pronóstico de hoy 2026-01-14 en Corrientes, Corrientes indica que no hay horas con probabilidad de lluvia.


 Que tengas un buen día! :)
mensaje enviado SM63cbb2141cd6d10f802f9143ed32da85
